In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from mnist import MNIST
mndata = MNIST('./data_mnist')
mndata.gz = True
images, labels = mndata.load_training()

C:\Users\DuyLe\miniforge3\envs\mnist\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
images = np.reshape(images, (60000, 1, 28 ,28))
labels = np.array(labels)
b = np.zeros((labels.size, labels.max() + 1))
b[np.arange(labels.size), labels] = 1
labels = b
images = images/255
labels = torch.tensor(labels).to("cuda")
images = torch.tensor(images).to("cuda")
images = images.to(dtype=torch.float32)

In [ ]:
class Linear_Mapping(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # self.weight=torch.nn.parameter.Parameter(torch.randn(784,10))
        # self.bias=torch.nn.parameter.Parameter(torch.randn(10))
        self.linear = torch.nn.Linear(784, 10)
    def forward(self,inp):
        inp = torch.flatten(inp, start_dim=1)
        # return (torch.matmul(inp, self.weight) + self.bias)
        return self.linear(inp)

model = Linear_Mapping().to("cuda")
list(model.parameters())

[Parameter containing:
 tensor([[-0.0083, -0.0027, -0.0286,  ..., -0.0245, -0.0344, -0.0134],
         [-0.0154,  0.0335,  0.0318,  ...,  0.0244,  0.0168, -0.0125],
         [ 0.0283, -0.0066,  0.0133,  ...,  0.0103, -0.0223, -0.0309],
         ...,
         [-0.0215,  0.0188,  0.0101,  ...,  0.0329, -0.0337, -0.0097],
         [-0.0349, -0.0291, -0.0049,  ..., -0.0095,  0.0152, -0.0271],
         [ 0.0203,  0.0267,  0.0041,  ...,  0.0320,  0.0124, -0.0123]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([-0.0127,  0.0201,  0.0109, -0.0203, -0.0172, -0.0307, -0.0332,  0.0139,
          0.0253,  0.0342], device='cuda:0', requires_grad=True)]

In [ ]:
epoch = 1000
losslist = []
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
for i in range(epoch):
    sumloss = 0
    model.train()
    for g,l in zip(images,labels): 
        g = g.reshape(1,28,28)
        output = model(g)
        sftm = torch.softmax(output, dim=1)
        loss = torch.sum(torch.pow(torch.abs(sftm-l),2))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sumloss = sumloss + loss
    meanloss = sumloss/60000
    losslist.append(meanloss.cpu())
    print(i, meanloss)

0 tensor(0.1912, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
1 tensor(0.1617, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
2 tensor(0.1525, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
3 tensor(0.1469, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
4 tensor(0.1411, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
5 tensor(0.1396, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
6 tensor(0.1364, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
7 tensor(0.1359, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
8 tensor(0.1313, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
9 tensor(0.1324, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
10 tensor(0.1290, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
11 tensor(0.1289, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
12 tensor(0.1279, device='cuda:0', dtype=torch.flo

In [ ]:
epoch = 1000
losslist = []
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
ce_loss = nn.CrossEntropyLoss()
for i in range(epoch):
    sumloss = 0
    model.train()
    for g,l in zip(images,labels): 
        g = g.reshape(1,28,28)
        l = l.unsqueeze(0)
        output = model(g)
        sftm = torch.softmax(output, dim=1)
        loss = ce_loss(g,l)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sumloss = sumloss + loss
    meanloss = sumloss/60000
    losslist.append(meanloss.cpu())
    print(i, meanloss)

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1)
        
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(in_features=82944, out_features=10)
        self.relu = nn.ReLU()
    def forward(self, inp):
        output = self.conv1(inp)
        output = self.relu(output)
        output = self.conv2(output)
        output = self.relu(output)
        output = self.conv3(output)
        output = self.relu(output)
        output = self.conv4(output)
        output = self.relu(output)
        output = self.conv5(output)
        output = self.relu(output)
        output = self.flatten(output)
        output = self.dense(output)
        return output
    
model_CNN = CNN().to("cuda")

In [ ]:
images.shape

In [ ]:
epoch = 1000
losslist_CNN = []
optimizer = torch.optim.SGD(model_CNN.parameters(), lr=0.01)
for i in range(epoch):
    sumloss = 0
    model_CNN.train()
    for g,l in zip(images,labels):
        g = g.reshape(1, 1, 28, 28)
        output = model_CNN(g)
        sftm = torch.softmax(output, dim=1)
        loss = torch.sum(torch.pow(torch.abs(sftm-l),2))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sumloss = sumloss + loss
    meanloss = sumloss/60000
    losslist_CNN.append(meanloss.cpu())
    print(i, meanloss)

0 tensor(0.0809, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
1 tensor(0.0298, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
2 tensor(0.0225, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
3 tensor(0.0182, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
4 tensor(0.0142, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
5 tensor(0.0123, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
6 tensor(0.0121, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
7 tensor(0.0122, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
8 tensor(0.0108, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
9 tensor(0.0104, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
10 tensor(0.0088, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
11 tensor(0.0117, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
12 tensor(0.0160, device='cuda:0', dtype=torch.flo

KeyboardInterrupt: 